In [105]:
import numpy as np
np.set_printoptions(suppress=True)

In [136]:
def get_norm(U):
    return np.sqrt(np.dot(U, U))

def unit_shorten(U):
    norm = get_norm(U)
    return U * 1/norm

def projection(U, V):
    U = U.flatten()
    V = V.flatten()
    dot = np.dot(U, V)
    norm = get_norm(U) ** 2
    proj = dot/norm * U
    return proj

def check_convergence(A, B):
    D = A - B
    for i in D.flatten():
        if i > 1e-6:
            return False
    return True

def qr_decompose(arr):
    m = arr.shape[0]
    n = arr.shape[1]
    orthmat = arr.copy()
    upper = np.zeros(arr.shape)

    # set 1, 1 element of R
    upper[0, 0] = get_norm(arr[:, 0])

    for i in range(orthmat.shape[1]):
        v = orthmat[:, i].reshape(-1,)
        if i > 0:
            residual_sum = np.zeros(v.shape)
            U = np.split(orthmat[:, :i], i, axis=1)
            for u in U:
                residual = projection(u, v)
                residual_sum += residual
            v -= residual_sum
            upper[i, i] = get_norm(v)
        orthmat[:, i] = unit_shorten(v)
# calculate the entries in R that are above the diagonal
    for i in range(m):
        for j in range(n):
            if j > i:
                upper[i, j] = np.dot(arr[:, j], orthmat[:, i])
    return orthmat, upper


In [168]:
a = np.array([[1,2,3], [3, 1, 4], [5, 1, 0]], dtype=np.float64)
b = np.array([[1,2,3], [3, 1, 4], [5, 1, 0]], dtype=np.float64).transpose()
A = a @ b

In [171]:
def main():
    iters = 0
    A_next = A.copy()  # Initialize A_next with a copy of A
    A_prev = np.eye(A.shape[0], dtype=np.float64)

    while not check_convergence(A_next, A_prev):
        A_prev = A_next  # Store the current A_next as A_prev before updating
        Q, R = qr_decompose(A_next)
        A_next = R @ Q
        iters += 1
        if iters > 999:
            print('Warning: 999 iterations reached without convergence!')
            break

    eigenvalues = [A_next[i, i] for i in range(A.shape[0])]
    print('Eigenvalues are calculated as: ', eigenvalues)

if __name__ == '__main__':
    main()

Eigenvalues are calculated as:  [50.23128322004287, 14.536124006136003, 1.2325927738211266]
